# Parallel / Intersecting Bus Routes to the SHN

In [1]:
import branca
import geopandas as gpd
import intake
import ipywidgets as widgets
import pandas as pd

from IPython.display import Markdown, HTML

import setup_corridors_stats
from create_parallel_corridors import IMG_PATH, DATA_PATH
from shared_utils import map_utils, geography_utils
from shared_utils import calitp_color_palette as cp

catalog = intake.open_catalog("./*.yml")

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(
E0428 20:15:15.504573159    3431 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
E0428 20:15:17.702083059    3431 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies


In [2]:
operator_stats, hwy_stats = setup_corridors_stats.aggregated_transit_hwy_stats()

In [ ]:
# Rename columns to better display
rename_dict = {
    "itp_id": "ITP ID",
    "pct_parallel": "% parallel",
    "unique_route_id": "# Unique Bus Routes",
    "num_parallel": "# Parallel Bus Routes",
    "pct_competitive": "% competitive",
    "num_competitive": "# Competitive Bus Routes",
}

In [ ]:
def display_operator_stats(operator_stats, select_col = "itp_id", operator_name = 182):
    operator_stats = operator_stats[operator_stats[select_col]==operator_name]
    # Style table
    operator_table = (operator_stats.sort_values(["pct_parallel", "pct_competitive"], 
                                                 ascending=[False, False])
                      .rename(columns = rename_dict)
                      .drop(columns = ["hwy_list"])
                      .style.format({'% parallel': '{:,.1%}', 
                                     '% competitive': '{:,.1%}',
                                    })
                      .hide(axis="index")
                      .to_html()
    )    
    
    display(Markdown(f"#### Summary Stats for ITP ID: {operator_name}"))
    display(HTML(operator_table))

In [ ]:
def make_transit_map(operator_df, hwy_df):    
    def data_to_plot(df):
        keep_cols = ["itp_id", "route_id", 
                     "Route", "County", "District", "RouteType",
                     "pct_route", "pct_highway", "parallel",
                     "geometry"
                    ]
        #df = df[keep_cols].reset_index(drop=True)
        df = df.assign(
            geometry = df.geometry.buffer(150).simplify(tolerance=100),
        )

        # Use simplify to make gdf smaller
        # folium map is creating too large of an HTML file to check in

        return df

    to_map = data_to_plot(operator_df)
    print(to_map.columns)
    
    # Set various components for map
    hwys_popup_dict = {
        "Route": "Highway Route",
        "RouteType": "Route Type",
        "County": "County"   
    }

    transit_popup_dict = {
        "itp_id": "Operator ITP ID",
        "route_id": "Route ID",
        "pct_route": "% overlapping route",
        "pct_highway": "% overlapping highway",
    }

    hwys_color = branca.colormap.StepColormap(
        colors=["black", "gray"],
    )

    colorscale = branca.colormap.StepColormap(
        colors=[
            cp.CALITP_CATEGORY_BRIGHT_COLORS[0], #blue
            cp.CALITP_CATEGORY_BRIGHT_COLORS[1] # orange
        ],
    )
    
    # Instead of using county centroid, calculate centroid from transit_df
    # Otherwise, it's too zoomed out from where transit routes are
    transit_centroid = (to_map
                        .to_crs(geography_utils.WGS84).geometry.centroid
                        .iloc[0]
                       )

    LAYERS_DICT = {
        "Highways": {"df": hwy_df,
            "plot_col": "Route",
            "popup_dict": hwys_popup_dict, 
            "tooltip_dict": hwys_popup_dict,
            "colorscale": hwys_color,
        },
        "Transit Routes": {"df": to_map,
            "plot_col": "parallel",
            "popup_dict": transit_popup_dict, 
            "tooltip_dict": transit_popup_dict,
            "colorscale": colorscale,
        },
    }
    
    LEGEND_URL = (
        "https://github.com/cal-itp/data-analyses/raw/"
        "main/bus_service_increase/"
        "img/legend_intersecting_parallel.png"
    )
    
    LEGEND_DICT = {
        "legend_url": LEGEND_URL,
        "legend_bottom": 85,
        "legend_left": 5,
    }
     
    
    fig = map_utils.make_folium_multiple_layers_map(
        LAYERS_DICT,
        fig_width = 700, fig_height = 700, 
        zoom=11, 
        centroid = [round(transit_centroid.y,2), 
                    round(transit_centroid.x, 2)], 
        title=f"Parallel vs Intersecting Lines for {to_map.itp_id.iloc[0]}",
        legend_dict = LEGEND_DICT
    )
    
    display(fig)
    #fig.save(f"{IMG_PATH}parallel_{operator_name}.html")
    #print(f"{operator_name} map saved")

In [ ]:
def display_highway_stats(hwy_stats, select_col = "Route", hwy_name = 5):
    # Subset for highways
    hwy_stats = hwy_stats[hwy_stats[select_col]==hwy_name]
    hwy_table = (hwy_stats.sort_values("pct_parallel", ascending=False)
                 .rename(columns = rename_dict)
                 .reset_index(drop=True)
                 .style.format({'% parallel': '{:,.1%}'})
                 .hide(axis="index")
                 .to_html()
                )    
    
    display(Markdown(f"#### Summary Stats for Highway Route: {hwy_name}"))
    display(HTML(hwy_table))
    

In [ ]:
# Generalize selection column in ipywidget dropdown
# Be able to select operator or highway route, etc
def interactive_widget(select_col):
    gdf = catalog.parallel_or_intersecting_routes.read()
    gdf = gdf[(gdf.Route.notna()) & (gdf.itp_id.notna())]
    
    operator_stats, hwy_stats = setup_corridors_stats.aggregated_transit_hwy_stats()    
    
    dropdown_labels = {
        "itp_id": "ITP ID",
        "Route": "Hwy Route",
    }
    
    dropdown = widgets.Dropdown(
        description=f"{dropdown_labels[select_col]}",
        options=gdf[select_col].astype(int).sort_values().unique().tolist(),
    )
    output = widgets.Output()

    display(dropdown)
    display(output)

    def on_selection(*args):
        output.clear_output()
        with output:
            if select_col=="Route":
                display_highway_stats(hwy_stats, select_col, dropdown.value)
            elif select_col=="itp_id":
                # Grab the full df and subset to specific
                # Break out the map-making function so the relevant highways can be extracted
                operator_subset = operator_stats[operator_stats[select_col] == dropdown.value]
                
                # Rethink this function...operator_df right now doesn't include competitive info
                operator_df, hwys_df = setup_corridors_stats.routes_highways_geom_for_operator(operator_subset)
                
                display_operator_stats(operator_stats, select_col, dropdown.value)
                make_transit_map(operator_df, hwys_df)

    dropdown.observe(on_selection, names="value")
    on_selection()

In [ ]:
display(Markdown("### Parallel Corridor Stats by Operator"))
interactive_widget("itp_id")

In [ ]:
display(Markdown("### Parallel Corridor Stats by Highway Route"))
interactive_widget("Route")